In [2]:
import os
import json
import re
import shutil
input_folder = "encoded"
output_folder = "done"
import torch
import torch.nn.functional as F
import textwrap

In [3]:
# Use list comprehension to create a list of file paths
verilog_files = [os.path.join(input_folder, filename) for filename in os.listdir(input_folder) if filename.endswith('.txt')]

# Print the list of file paths
print(len(verilog_files))
print(type(verilog_files[0]))
print(verilog_files[0])


290
<class 'str'>
encoded\adder11.txt


In [23]:
with open("labels_dict.txt", 'r') as file:
    data = file.read()
data_dict = json.loads(data)
for key, value in data_dict.items():
    print(key, value)

adder 14
ALU 9
and 25
comparator 24
decoder 26
encoder 23
mult 24
mux 26
nand 20
nor 16
not 11
or 24
pe 16
sub 9
xnor 23


In [16]:
num_connections_list = []
size_list = []
num_nodes_files_list = []
num_edges_files_list = []
for file in verilog_files:
    with open(file, "r") as file:
        loaded_data = json.load(file)
        num_nodes_files_list.append(len(loaded_data[0]))
        num_edges_files_list.append(len(loaded_data[1][0]))
        for sublist in loaded_data[0]:
            if sublist[2] == 0:
                print(file.name)
                continue
            num_connections_list.append(sublist[2])
            size_list.append(sublist[3])

encoded\ALU10.txt
encoded\ALU13.txt
encoded\decoder30.txt
encoded\encoder16.txt
encoded\encoder18.txt
encoded\encoder6.txt
encoded\pe14.txt
encoded\pe15.txt
encoded\pe16.txt
encoded\pe22.txt
encoded\pe7.txt


In [17]:
average_connections = sum(num_connections_list) / len(num_connections_list)
max_connections = max(num_connections_list)
min_connections = min(num_connections_list)

print("Average connections: ", average_connections)
print("Max connections: ", max_connections)
print("Min connections: ", min_connections)

Average connections:  2.506062685884237
Max connections:  34
Min connections:  1


In [18]:
average_size = sum(size_list) / len(size_list)
max_size = max(size_list)
min_size = min(size_list)

print("Average node size: ", average_size)
print("Max node size: ", max_size)
print("Min node size: ", min_size)

Average node size:  2.578586135895676
Max node size:  32
Min node size:  1


In [19]:
avergae_num_nodes = sum(num_nodes_files_list) / len(num_nodes_files_list)
max_num_nodes = max(num_nodes_files_list)
min_num_nodes = min(num_nodes_files_list)

print("Average number of nodes: ", avergae_num_nodes)
print("Max number of nodes: ", max_num_nodes)
print("Min number of nodes: ", min_num_nodes)

Average number of nodes:  15.110344827586207
Max number of nodes:  165
Min number of nodes:  3


In [20]:
average_edges = sum(num_edges_files_list) / len(num_edges_files_list)
max_edges = max(num_edges_files_list)
min_edges = min(num_edges_files_list)

print("Average number of edges: ", average_edges)
print("Max number of edges: ", max_edges)
print("Min number of edges: ", min_edges)

Average number of edges:  18.575862068965517
Max number of edges:  208
Min number of edges:  2


In [40]:
sum_nodes = sum(num_nodes_files_list)
sum_nodes

4382

In [21]:
# write global statistics to file

In [137]:
files_dict = dict()
for key in data_dict.keys():
    num_nodes = 0
    num_edges = 0
    num_files = 0
    # node_sizes = []
    for file_path in verilog_files:
        st = file_path.split("\\")[1].split(".")[0]
        # print(st)
        pattern = f"^({key})\d+"
        # print(pattern)
        if re.match(pattern, st):
            # print("hi")
            num_files += 1
            with open(file_path, "r") as file:
                loaded_data = json.load(file)
                num_nodes += len(loaded_data[0])
                num_edges += len(loaded_data[1][0])
                # for sublist in loaded_data[0]:
                #     node_sizes.append(sublist[3])
                
    files_dict[key] = {
        "num_nodes": num_nodes,
        "num_edges": num_edges,
        "average_nodes": round(num_nodes / num_files,2),
        "average_edges": round(num_edges / num_files,2),
        "num_files": num_files,
        "percentage_nodes" : round(num_nodes / sum_nodes * 100,2),
        "percentage_edges" : round(num_edges / sum(num_edges_files_list) * 1000,2),
        "percentage_files" : round(num_files / len(verilog_files) * 100,2)
    }
# files_dict

In [138]:
files_dict["adder"]

{'num_nodes': 594,
 'num_edges': 877,
 'average_nodes': 42.43,
 'average_edges': 62.64,
 'num_files': 14,
 'percentage_nodes': 13.56,
 'percentage_edges': 162.8,
 'percentage_files': 4.83}

In [139]:
files_dict["ALU"]

{'num_nodes': 250,
 'num_edges': 378,
 'average_nodes': 27.78,
 'average_edges': 42.0,
 'num_files': 9,
 'percentage_nodes': 5.71,
 'percentage_edges': 70.17,
 'percentage_files': 3.1}

In [140]:
files_dict["and"]

{'num_nodes': 143,
 'num_edges': 142,
 'average_nodes': 5.72,
 'average_edges': 5.68,
 'num_files': 25,
 'percentage_nodes': 3.26,
 'percentage_edges': 26.36,
 'percentage_files': 8.62}

In [141]:
files_dict["comparator"]

{'num_nodes': 409,
 'num_edges': 502,
 'average_nodes': 17.04,
 'average_edges': 20.92,
 'num_files': 24,
 'percentage_nodes': 9.33,
 'percentage_edges': 93.19,
 'percentage_files': 8.28}

In [142]:
files_dict["decoder"]

{'num_nodes': 780,
 'num_edges': 1028,
 'average_nodes': 30.0,
 'average_edges': 39.54,
 'num_files': 26,
 'percentage_nodes': 17.8,
 'percentage_edges': 190.83,
 'percentage_files': 8.97}

In [143]:
files_dict["encoder"]

{'num_nodes': 400,
 'num_edges': 439,
 'average_nodes': 17.39,
 'average_edges': 19.09,
 'num_files': 23,
 'percentage_nodes': 9.13,
 'percentage_edges': 81.49,
 'percentage_files': 7.93}

In [144]:
files_dict["mult"]

{'num_nodes': 371,
 'num_edges': 463,
 'average_nodes': 15.46,
 'average_edges': 19.29,
 'num_files': 24,
 'percentage_nodes': 8.47,
 'percentage_edges': 85.95,
 'percentage_files': 8.28}

In [145]:
files_dict["mux"]

{'num_nodes': 419,
 'num_edges': 538,
 'average_nodes': 16.12,
 'average_edges': 20.69,
 'num_files': 26,
 'percentage_nodes': 9.56,
 'percentage_edges': 99.87,
 'percentage_files': 8.97}

In [146]:
files_dict["nand"]

{'num_nodes': 137,
 'num_edges': 135,
 'average_nodes': 6.85,
 'average_edges': 6.75,
 'num_files': 20,
 'percentage_nodes': 3.13,
 'percentage_edges': 25.06,
 'percentage_files': 6.9}

In [147]:
files_dict["nor"]

{'num_nodes': 120,
 'num_edges': 104,
 'average_nodes': 7.5,
 'average_edges': 6.5,
 'num_files': 16,
 'percentage_nodes': 2.74,
 'percentage_edges': 19.31,
 'percentage_files': 5.52}

In [148]:
files_dict["not"]

{'num_nodes': 57,
 'num_edges': 46,
 'average_nodes': 5.18,
 'average_edges': 4.18,
 'num_files': 11,
 'percentage_nodes': 1.3,
 'percentage_edges': 8.54,
 'percentage_files': 3.79}

In [149]:
files_dict["or"]

{'num_nodes': 147,
 'num_edges': 129,
 'average_nodes': 6.12,
 'average_edges': 5.38,
 'num_files': 24,
 'percentage_nodes': 3.35,
 'percentage_edges': 23.95,
 'percentage_files': 8.28}

In [150]:
files_dict["pe"]

{'num_nodes': 264,
 'num_edges': 317,
 'average_nodes': 16.5,
 'average_edges': 19.81,
 'num_files': 16,
 'percentage_nodes': 6.02,
 'percentage_edges': 58.85,
 'percentage_files': 5.52}

In [151]:
files_dict["sub"]

{'num_nodes': 131,
 'num_edges': 147,
 'average_nodes': 14.56,
 'average_edges': 16.33,
 'num_files': 9,
 'percentage_nodes': 2.99,
 'percentage_edges': 27.29,
 'percentage_files': 3.1}

In [152]:
files_dict["xnor"]

{'num_nodes': 160,
 'num_edges': 142,
 'average_nodes': 6.96,
 'average_edges': 6.17,
 'num_files': 23,
 'percentage_nodes': 3.65,
 'percentage_edges': 26.36,
 'percentage_files': 7.93}

In [156]:
nodes_sizes_dict = dict()
for key in data_dict.keys():
    node_sizes = []
    node_connections = []
    for file_path in verilog_files:
        st = file_path.split("\\")[1].split(".")[0]
        pattern = f"^({key})\d+"
        if re.match(pattern, st):
            with open(file_path, "r") as file:
                loaded_data = json.load(file)
                for sublist in loaded_data[0]:
                    node_sizes.append(sublist[3])
                    node_connections.append(sublist[2])
                
    nodes_sizes_dict[key] = {
        "node connections list": str(node_connections),
        "node size list" : str(node_sizes)
    }


In [158]:
nodes_sizes_dict["adder"]

{'node connections list': '[1, 1, 16, 16, 16, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 3, 1, 3, 3, 3, 3, 1, 3, 1, 3, 1, 3, 3, 3, 4, 1, 3, 1, 3, 1, 3, 3, 3, 4, 1, 3, 1, 3, 1, 3, 3, 3, 4, 1, 3, 1, 3, 1, 3, 3, 3, 4, 1, 3, 1, 3, 1, 3, 3, 3, 4, 1, 3, 1, 3, 1, 3, 3, 3, 4, 1, 3, 1, 3, 1, 3, 3, 3, 4, 1, 3, 1, 3, 1, 3, 3, 3, 4, 1, 3, 1, 3, 1, 3, 3, 3, 4, 1, 3, 1, 3, 1, 3, 3, 3, 4, 1, 3, 1, 3, 1, 3, 3, 3, 4, 1, 3, 1, 3, 1, 3, 3, 3, 4, 1, 3, 1, 3, 1, 3, 3, 3, 4, 1, 3, 1, 3, 1, 3, 3, 3, 4, 1, 3, 1, 3, 1, 3, 3, 3, 1, 1, 2, 2, 1, 6, 3, 1, 1, 1, 1, 6, 3, 1, 1, 1, 4, 1, 32, 32, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 1, 12, 13, 5, 2, 12, 13, 5, 2, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

In [ ]:
# features = input  output  reg  wire  operation  no_op  add  and  or  xor  xnor  nor  nand  not  case  concat  conditional  constVal  mult  mux  power  shift  subtr  
# index         0       1     2    3     4          5    6    7    8   9    10    11    12    13   14    15        16         17       18     19    20    21    22

In [163]:
encoded_files = [os.path.join(output_folder, filename) for filename in os.listdir(input_folder) if filename.endswith('.txt')]
encoded_files

['done\\adder11.txt',
 'done\\adder12.txt',
 'done\\adder13.txt',
 'done\\adder14.txt',
 'done\\adder15.txt',
 'done\\adder16.txt',
 'done\\adder17.txt',
 'done\\adder18.txt',
 'done\\adder19.txt',
 'done\\adder2.txt',
 'done\\adder20.txt',
 'done\\adder5.txt',
 'done\\adder6.txt',
 'done\\adder8.txt',
 'done\\ALU10.txt',
 'done\\ALU13.txt',
 'done\\ALU14.txt',
 'done\\ALU15.txt',
 'done\\ALU2.txt',
 'done\\ALU6.txt',
 'done\\ALU7.txt',
 'done\\ALU8.txt',
 'done\\ALU9.txt',
 'done\\and1.txt',
 'done\\and10.txt',
 'done\\and12.txt',
 'done\\and13.txt',
 'done\\and14.txt',
 'done\\and15.txt',
 'done\\and16.txt',
 'done\\and17.txt',
 'done\\and18.txt',
 'done\\and19.txt',
 'done\\and2.txt',
 'done\\and20.txt',
 'done\\and21.txt',
 'done\\and23.txt',
 'done\\and25.txt',
 'done\\and26.txt',
 'done\\and27.txt',
 'done\\and28.txt',
 'done\\and29.txt',
 'done\\and3.txt',
 'done\\and30.txt',
 'done\\and6.txt',
 'done\\and7.txt',
 'done\\and8.txt',
 'done\\and9.txt',
 'done\\comparator1.txt',
 '

In [167]:
nodes_files_info = dict()
input_count = 0
output_count = 0
reg_count = 0
wire_count = 0
subtr_count = 0
shift_count = 0
power_count = 0
mux_count = 0
mult_count = 0
constVal_count = 0
conditional_count = 0
concat_count = 0
case_count = 0
not_count = 0
nand_count = 0
nor_count = 0
xnor_count = 0
xor_count = 0
or_count = 0
and_count = 0
add_count = 0
for key in data_dict.keys():
    key_dict = dict()
    for file_path in encoded_files:
        st = file_path.split("\\")[1].split(".")[0]
        pattern = f"^({key})\d+"
        if re.match(pattern, st):
            with open(file_path, "r") as file:
                loaded_data = json.load(file)
                for sublist in loaded_data[0]:
                    # print(sublist)
                    # break
                    if sublist[0]: # input
                        key_dict["input"] = key_dict.get("input", 0) + 1
                        input_count +=1
                        
                    elif sublist[1]: #output
                        key_dict["output"] = key_dict.get("output", 0) + 1
                        output_count += 1
                        
                    elif sublist[2]: #reg
                        key_dict["reg"] = key_dict.get("reg", 0) + 1
                        reg_count +=1
                        
                    elif sublist[3]: #wire
                        key_dict["wire"] = key_dict.get("wire", 0) + 1
                        wire_count +=1
                        
                    elif sublist[4]: #operation
                        # key_dict["operation"] = key_dict.get("operation", 0) + 1
                        if sublist[5]:
                            key_dict["subtr"] = key_dict.get("subtr", 0) + 1
                            subtr_count += 1
                            
                        elif sublist[6]:
                            key_dict["shift"] = key_dict.get("shift", 0) + 1
                            shift_count +=1
                            
                        elif sublist[7]:
                            key_dict["power"] = key_dict.get("power", 0) + 1
                            power_count+=1
                            
                        elif sublist[8]:
                            key_dict["mux"] = key_dict.get("mux", 0) + 1
                            mux_count+=1
                            
                        elif sublist[9]:
                            key_dict["mult"] = key_dict.get("mult", 0) + 1
                            mult_count+=1
                            
                        elif sublist[10]:
                            key_dict["constVal"] = key_dict.get("constVal", 0) + 1
                            constVal_count+=1
                            
                        elif sublist[11]:
                            key_dict["conditional"] = key_dict.get("conditional", 0) + 1
                            conditional_count+=1
                            
                        elif sublist[12]:
                            key_dict["concat"] = key_dict.get("concat", 0) + 1
                            concat_count +=1
                            
                        elif sublist[13]:
                            key_dict["case"] = key_dict.get("case", 0) + 1
                            case_count+=1
                            
                        elif sublist[14]:
                            key_dict["not"] = key_dict.get("not", 0) + 1
                            not_count+=1
                            
                        elif sublist[15]:
                            key_dict["nand"] = key_dict.get("nand", 0) + 1
                            nand_count+=1
                            
                        elif sublist[16]:
                            key_dict["nor"] = key_dict.get("nor", 0) + 1
                            nor_count+=1
                            
                        elif sublist[17]:
                            key_dict["xnor"] = key_dict.get("xnor", 0) + 1
                            xnor_count+=1
                            
                        elif sublist[18]:
                            key_dict["xor"] = key_dict.get("xor", 0) + 1
                            xor_count+=1
                            
                        elif sublist[19]:
                            key_dict["or"] = key_dict.get("or", 0) + 1
                            or_count+=1
                            
                        elif sublist[20]:
                            key_dict["and"] = key_dict.get("and", 0) + 1
                            and_count+=1
                            
                        elif sublist[21]:
                            key_dict["add"] = key_dict.get("add", 0) + 1
                            add_count+=1
                            
                        else: 
                            print("eh el habal da")
                            
    nodes_files_info[key] = key_dict
                
    
# files_dict

In [170]:
print("Input Nodes Count = ", input_count)
print("Output Nodes Count = ", output_count)
print("Reg Nodes Count = ", reg_count)
print("Wire Nodes Count = ", wire_count)
print()
print("Subtr Nodes Count = ", subtr_count)
print("Shift Nodes Count = ", shift_count)
print("Power Nodes Count = ", power_count)
print("Mux Nodes Count = ", mux_count)
print("Mult Nodes Count = ", mult_count)
print("ConstVal Nodes Count = ", constVal_count)
print("Conditional Nodes Count = ", conditional_count)
print("Concat Nodes Count = ", concat_count)
print("Case Nodes Count = ", case_count)
print("Not Nodes Count = ", not_count)
print("Nand Nodes Count = ", nand_count)
print("Nor Nodes Count = ", nor_count)
print("Xnor Nodes Count = ", xnor_count)
print("Xor Nodes Count = ", xor_count)
print("Or Nodes Count = ", or_count)
print("And Nodes Count = ", and_count)
print("Add Nodes Count = ", add_count)

Input Nodes Count =  660
Output Nodes Count =  387
Reg Nodes Count =  59
Wire Nodes Count =  72

Subtr Nodes Count =  8
Shift Nodes Count =  27
Power Nodes Count =  0
Mux Nodes Count =  301
Mult Nodes Count =  7
ConstVal Nodes Count =  1025
Conditional Nodes Count =  364
Concat Nodes Count =  64
Case Nodes Count =  72
Not Nodes Count =  251
Nand Nodes Count =  9
Nor Nodes Count =  3
Xnor Nodes Count =  6
Xor Nodes Count =  131
Or Nodes Count =  279
And Nodes Count =  623
Add Nodes Count =  23


In [171]:
nodes_files_info["adder"]

{'output': 28,
 'input': 50,
 'constVal': 120,
 'mux': 24,
 'conditional': 64,
 'and': 161,
 'case': 6,
 'concat': 6,
 'xor': 64,
 'or': 68,
 'wire': 3}

In [172]:
nodes_files_info["ALU"]

{'output': 9,
 'input': 28,
 'and': 14,
 'or': 11,
 'xor': 11,
 'not': 9,
 'mux': 40,
 'constVal': 52,
 'conditional': 43,
 'add': 7,
 'subtr': 8,
 'shift': 10,
 'case': 3,
 'reg': 1,
 'wire': 2}

In [173]:
nodes_files_info["comparator"]

{'output': 56,
 'input': 49,
 'wire': 32,
 'constVal': 62,
 'not': 38,
 'xor': 20,
 'and': 36,
 'or': 16,
 'xnor': 5,
 'mux': 27,
 'conditional': 47,
 'concat': 10,
 'nor': 2,
 'reg': 5,
 'case': 4}

In [174]:
nodes_files_info["decoder"]

{'output': 40,
 'input': 57,
 'constVal': 231,
 'case': 12,
 'mux': 31,
 'not': 126,
 'and': 216,
 'concat': 2,
 'shift': 17,
 'conditional': 33,
 'wire': 6,
 'nand': 8}

In [175]:
nodes_files_info["encoder"]

{'output': 36,
 'input': 70,
 'or': 64,
 'constVal': 141,
 'case': 10,
 'and': 13,
 'mux': 31,
 'conditional': 24,
 'concat': 4,
 'not': 4}

In [176]:
nodes_files_info["mult"]

{'output': 24,
 'input': 56,
 'mult': 7,
 'reg': 10,
 'constVal': 116,
 'case': 9,
 'concat': 15,
 'and': 86,
 'xor': 4,
 'add': 16,
 'wire': 10,
 'not': 18}

In [177]:
nodes_files_info["mux"]

{'output': 26,
 'input': 95,
 'reg': 5,
 'case': 9,
 'constVal': 90,
 'conditional': 91,
 'mux': 96,
 'not': 3,
 'wire': 1,
 'and': 2,
 'or': 1}

In [178]:
nodes_files_info["pe"]

{'output': 31,
 'input': 49,
 'constVal': 32,
 'mux': 15,
 'conditional': 15,
 'wire': 9,
 'or': 78,
 'and': 22,
 'not': 8}

In [179]:
nodes_files_info["sub"]

{'output': 18,
 'input': 24,
 'xor': 13,
 'not': 10,
 'or': 7,
 'and': 12,
 'constVal': 32,
 'case': 6,
 'concat': 6,
 'wire': 3}

In [180]:
nodes_files_info["and"]

{'output': 25,
 'input': 39,
 'and': 29,
 'reg': 7,
 'constVal': 22,
 'case': 5,
 'concat': 10,
 'conditional': 5,
 'mux': 1}

In [181]:
nodes_files_info["or"]

{'output': 24,
 'input': 36,
 'or': 20,
 'reg': 8,
 'constVal': 29,
 'conditional': 9,
 'and': 3,
 'mux': 6,
 'case': 4,
 'concat': 8}

In [182]:
nodes_files_info["not"]

{'output': 11,
 'input': 11,
 'not': 4,
 'constVal': 17,
 'case': 4,
 'reg': 4,
 'mux': 3,
 'conditional': 3}

In [183]:
nodes_files_info["nand"]

{'output': 20,
 'input': 31,
 'and': 25,
 'not': 14,
 'concat': 3,
 'reg': 6,
 'wire': 4,
 'constVal': 18,
 'mux': 6,
 'nand': 1,
 'conditional': 6,
 'or': 3}

In [184]:
nodes_files_info["nor"]

{'output': 16,
 'input': 27,
 'or': 11,
 'not': 8,
 'constVal': 27,
 'conditional': 11,
 'mux': 8,
 'reg': 5,
 'nor': 1,
 'wire': 2,
 'and': 4}

In [185]:
nodes_files_info["xnor"]

{'output': 23,
 'input': 38,
 'not': 9,
 'xor': 19,
 'reg': 8,
 'constVal': 36,
 'conditional': 13,
 'mux': 13,
 'xnor': 1}

In [ ]:
# notes
# remove power
# remove operation and no operation encoder
